<a href="https://colab.research.google.com/github/VictorHo1114/Medical-Chatbot/blob/main/RobotMedical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install unsloth
# 安裝最新 Unsloth 依賴
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install opencc-python-reimplemented # 用於簡轉繁

In [ ]:
import os
from google.colab import drive
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from opencc import OpenCC
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# 0. 確保 Google Drive 已掛載 (最優先執行)
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 1. 設定模型參數 (統一在這裡設定)
max_seq_length = 1024 # ✂️ 關鍵：直接鎖死 1024，確保跑得動
dtype = None
load_in_4bit = True

# 2. 載入 Qwen 2.5 3B (只載入這一次！)
print("🔄 正在載入模型...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-3B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 3. 加上 LoRA 適配器
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 64,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

# 4. 準備數據與繁體轉換
print("📚 正在處理數據集 (含繁體轉換)...")
cc = OpenCC('s2twp') # 簡轉繁 (台灣正體)

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
你是一位專業的醫療AI助手。請詳細進行推理並回答使用者的醫療問題。
**請務必使用繁體中文（Traditional Chinese）回答。**

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for instruction, cot, output in zip(instructions, cots, outputs):
        # 組合推理鏈 (CoT) 與最終結論
        full_response = f"{cot}\n\n結論：{output}"

        # --- ⚡ 即時繁體轉換 ---
        instruction_tc = cc.convert(instruction)
        full_response_tc = cc.convert(full_response)

        text = alpaca_prompt.format(instruction_tc, full_response_tc) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

# 載入數據集 (記得選 "zh")
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "zh", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True)

# 5. 設定訓練參數 (極速版)
print("⚙️ 設定訓練參數...")
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length, # 這裡會自動使用上面設定的 1024
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 4, # 長度減半後，Batch 4 應該跑得動
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 200,         # 只跑 200 步
        learning_rate = 3e-4,    # 較高的學習率
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,

        # --- 💾 存檔保險 ---
        output_dir = "/content/drive/MyDrive/Medical_Qwen_Checkpoints",
        save_strategy = "steps",
        save_steps = 50,
        save_total_limit = 2,
    ),
)

# 6. 開始訓練
print("🚀 開始極速訓練！")
trainer_stats = trainer.train()

In [ ]:
import os

# 設定你剛剛代碼中的存檔路徑 (如果不確定，我們先搜整個 Checkpoints 資料夾)
# 根據你上一段代碼，路徑應該是這個：
checkpoints_path = "/content/drive/MyDrive/Medical_Qwen_Checkpoints"

print(f"📂 正在檢查路徑：{checkpoints_path}")

if os.path.exists(checkpoints_path):
    files = os.listdir(checkpoints_path)
    print(f"✅ 找到了！資料夾內有 {len(files)} 個物件。")
    print("內容包含：", files)

    # 檢查是否有最新的 checkpoint (例如 checkpoint-200)
    checkpoints = [f for f in files if f.startswith("checkpoint-")]
    if checkpoints:
        print(f"🏆 成功存檔的檢查點：{sorted(checkpoints)}")
    else:
        print("⚠️ 有資料夾，但沒看到 checkpoint-XXX，請檢查內容。")
else:
    print("❌ 沒找到路徑！可能是路徑打錯，或還沒同步。")

In [ ]:
!jupyter nbconvert --ClearOutputPreprocessor.enabled=True --inplace your_notebook.ipynb

In [ ]:
# 1. 切換到推理模式
FastLanguageModel.for_inference(model)

# 2. 準備問題 (測試它的醫療知識與繁體能力)
question = "醫生，我最近早上起床都會頭暈，而且手指尖有點麻麻的，這是中風的前兆嗎？我很擔心。"

# 3. 格式化輸入
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
你是一位專業的醫療AI助手。請詳細進行推理並回答使用者的醫療問題。
**請務必使用繁體中文（Traditional Chinese）回答。**

### Input:
{}

### Response:
"""

# 4. 生成回答
inputs = tokenizer(
    [alpaca_prompt.format(question)],
    return_tensors = "pt"
).to("cuda")

print("🤖 你的專屬醫療 AI 正在思考中...\n" + "="*30)

outputs = model.generate(
    **inputs,
    max_new_tokens = 512, # 讓它多講一點
    use_cache = True,
    temperature = 0.7,
)

# 5. 解碼並顯示
result = tokenizer.batch_decode(outputs)[0]
# 只顯示回答的部分
print(result.split("### Response:\n")[-1].replace("<|endoftext|>", ""))